In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import json
import logging
import os

logging.basicConfig(level=logging.INFO)
assert os.path.exists("../html")

LABELS = [
    "bugfix",
    "feature",
    "documentation",
    "performance",
    "enhancement",
    "maintenance",
]

In [3]:
from napari_dashboard.db_schema.base import Base
from napari_dashboard.db_schema.github import PullRequests
from napari_dashboard.db_update.github import (
    save_issues,
    save_pull_requests,
    save_stars,
    setup_cache,
)
from napari_dashboard.db_update.util import JSONEncoder
from napari_dashboard.gen_stat.github import (
    calc_stars_per_day_cumulative,
    count_recent_pull_requests_by_label,
    generate_basic_stats,
    get_contributors,
)

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

In [5]:
engine = create_engine("sqlite:///../sample.db")
Base.metadata.create_all(engine)

In [6]:
setup_cache()

In [8]:
with Session(engine) as session:
    save_stars("napari", "napari", session)
    save_stars("napari", "docs", session)

INFO:root:Already saved 2069 stars for napari/napari
INFO:root:Already saved 9 stars for napari/docs


In [9]:
with Session(engine) as session:
    save_pull_requests("napari", "napari", session)
    save_pull_requests("napari", "docs", session)

100%|███████████████████████████████████████| 3934/3934 [00:57<00:00, 68.06it/s]
INFO:root:Saved 0 pull requests for napari/napari
100%|█████████████████████████████████████████| 251/251 [00:06<00:00, 39.33it/s]
INFO:root:Saved 0 pull requests for napari/docs


In [7]:
with Session(engine) as session:
    save_issues("napari", "napari", session)
    save_issues("napari", "docs", session)

100%|█████████████████████████████████████| 6808/6808 [1:08:59<00:00,  1.64it/s]
INFO:root:Saved 6808 issues for napari/napari
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 419/419 [04:21<00:00,  1.60it/s]
INFO:root:Saved 419 issues for napari/docs


In [8]:
with Session(engine) as session:
    res = calc_stars_per_day_cumulative("napari", "napari", session)
with open("../html/stars.json", "w") as f:
    json.dump(res, f, cls=JSONEncoder)

In [22]:
with Session(engine) as session:
    s = get_contributors("napari", "napari", session)

[('2dx', 2),
 ('AbigailMcGovern', 3),
 ('AhmetCanSolak', 28),
 ('AmirAflak', 1),
 ('AndrewAnnex', 1),
 ('BhavyaC16', 1),
 ('Carreau', 207),
 ('Chris-N-K', 1),
 ('ClementCaporal', 1),
 ('Cryaaa', 1),
 ('Czaki', 417),
 ('DanGonite57', 1),
 ('DenisSch', 1),
 ('DragaDoncila', 70),
 ('Fifourche', 1),
 ('GenevieveBuckley', 39),
 ('HagaiHargil', 4),
 ('JoOkuma', 20),
 ('LucaMarconato', 1),
 ('MBPhys', 1),
 ('MaksHess', 1),
 ('Mishrasubha', 1),
 ('NHPatterson', 2),
 ('Nadalyn-CZI', 2),
 ('NicolasCARPi', 1),
 ('RDIL', 2),
 ('RobAnKo', 1),
 ('UmbWill', 1),
 ('VolkerH', 9),
 ('Zac-HD', 1),
 ('aaristov', 1),
 ('aeisenbarth', 2),
 ('aganders3', 14),
 ('akuten1298', 1),
 ('alexdesiqueira', 1),
 ('alisterburt', 57),
 ('andy-sweet', 110),
 ('arokem', 2),
 ('boonepeter', 5),
 ('brisvag', 109),
 ('bryantChhun', 2),
 ('cajongonzales', 1),
 ('camlloyd', 1),
 ('cgohlke', 4),
 ('chili-chiu', 9),
 ('clbarnes', 2),
 ('cnstt', 7),
 ('codemonkey800', 6),
 ('constantinpape', 1),
 ('ctr26', 1),
 ('ctrueden', 1),


In [9]:
with Session(engine) as session:
    print(
        generate_basic_stats(
            "napari",
            "napari",
            session,
            datetime.datetime(year=2024, month=1, day=1),
        )
    )

{'total_pull_requests': 3381, 'merged_pull_requests': 3381, 'open_pull_requests': 94, 'new_merged_pull_requests': 198, 'new_opened_pull_requests': 52}


In [10]:
with Session(engine) as session:
    print(
        generate_basic_stats(
            "napari",
            "napari",
            session,
            datetime.datetime(year=2024, month=1, day=1),
        )
    )

{'total_pull_requests': 3381, 'merged_pull_requests': 3287, 'open_pull_requests': 94, 'new_merged_pull_requests': 198, 'new_opened_pull_requests': 52}


In [23]:
with Session(engine) as session:
    print(
        session.query(PullRequests)
        .filter(PullRequests.pull_request == 9896)
        .all()
    )

[]


In [9]:
with Session(engine) as session:
    print(
        count_recent_pull_requests_by_label(
            "napari",
            "napari",
            session,
            datetime.datetime(year=2024, month=1, day=1),
            LABELS,
        )
    )

{'bugfix': 39, 'feature': 8, 'documentation': 10, 'performance': 4, 'enhancement': 9, 'maintenance': 135}


(datetime.datetime(2024, 5, 13, 12, 30, 49, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 5, 13, 12, 30, 49, tzinfo=datetime.timezone.utc))

5000